# Project: Sensitivity of coupon-bearing Treasury notes and bonds
Malkiel proposed five theorems that govern the price of fixed-income debt securities, e.g., Treasury Bills, Notes, and Bonds, as a function of the duration, yield, and coupon (interest) rate  values:

> [Malkiel, B. G. (1962). Expectations, Bond Prices, and the Term Structure of Interest Rates. The Quarterly Journal of Economics, 76(2), 197–218. https://doi.org/10.2307/1880816](https://www.jstor.org/stable/1880816)

In this project, let's develop a simulation to validate `Theorem 4` and `Theorem 5` of Malkiel
* `Theorem 4`: Price movements resulting from equal absolute increases and decreases in yield are asymmetric; i.e., decreasing yields raise bond prices more than the same increase in yields lowers prices.
* `Theorem 5`: The higher the coupon carried by the bond, the smaller the percentage price fluctuation for a given change in yield

## Learning objectives
The objective of this project is to familiarize students with computing the prices of Treasury Bills, Notes, and Bonds and to compare simulations of the change in the prices of these instruments with the five theorems proposed by Malkiel:

* __Objective 1__: Compute the price, visualize the cashflows for a coupon-bearing Treasury note
    * `TODO`: Build an instance of `MyUSTreasuryCouponSecurityModel`, and compute the price and other data for the note
        * `Check`: Are the computed and observed note prices `similar`?
    * `TODO`: Visualize the `nominal,` `discounted,` and `cumulative` cash flow
* __Objective 2__: Simulate Theorem 4 of Malkiel for a coupon-bearing Treasury note
    * `Discussion question`: Do your simulations support `Theorem 4?`
        1. Do your `Theorem 4` simulations support Malkiel's hypothesis, i.e., that the price change is asymmetric?
        2. Would you expect the price asymmetry to increase or decrease with the duration of the note or bond?
* __Objective 3__: Simulate Theorem 5 of Malkiel for a coupon-bearing Treasury note
    * `Discussion question`: Do your simulations support `Theorem 5?`
        * `Hint` compare the center square with the corners; what do the differences in value suggest?

## Setup
We load the [VLQuantitativeFinancePackage.jl](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl) package and several other external [Julia](https://julialang.org/downloads/https://julialang.org/downloads/) packages, as well as some helper code and custom Types (see below), by calling the [include(...)](https://docs.julialang.org/en/v1/manual/code-loading/https://docs.julialang.org/en/v1/manual/code-loading/) command on the file `Include.jl`:

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-131/module-2/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CH

### Packages
In addition to [VLQuantitativeFinancePackage.jl](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl), the `Include.jl` file loads several external packages that we will use for our exercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table, and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general-purpose data science tool.
* [Plots.jl](https://docs.juliaplots.org/stable/) is a plotting library in [Julia](https://julialang.org). We'll use this library for visualization
* The [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl) provides functions to display data in matrices in a human-readable format
    * This package exports the `pretty_table(...)` function which we use heavily

### Types
In addition to external packages, `Include.jl` also loads some [problem-specific custom types](https://docs.julialang.org/en/v1/manual/functions/) that will be helpful for the analysis of Treasury instrument pricing. 
* `MyUSTreasuryCouponSecurityModel` is a [mutable type](https://docs.julialang.org/en/v1/manual/types/#Mutable-Composite-Types) holding the par value $V_{P}$, the duration $T$, the effective market rate of interest rate $\bar{r}$, the coupoun rate $c$ and the number of coupon payments per year $\lambda$ for treasury note and bonds that pay a coupoun. You construct a `MyUSTreasuryCouponSecurityModel` instance using the `build` method described below.
* `DiscreteCompoundingModel` and `ContinuousCompoundingModel` are [immutable types](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) that let your code know which compounding model to use.

### Functions
`Include.jl` loads the following [Julia functions](https://docs.julialang.org/en/v1/manual/functions/) into the notebook:

`MyTreasuryBillDataSet() -> DataFrame`
> The `MyTreasuryBillDataSet() -> DataFrame` loads a United States Treasury auction dataset, which we'll use in the example. The auction data is returned as a `DataFrame`, which is exported by the [DataFrames.jl package](https://dataframes.juliadata.org/stable/)

`build(model::Type{MyUSTreasuryCouponSecurityModel}, data::NamedTuple) -> MyUSTreasuryCouponSecurityModel`
> This function takes information in the `data` [NamedTuple](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) argument (the par value, the number of compounding periods per year, the effective interest rate, the coupon rate, and the duration of the note or bond) and returns an instance of the `MyUSTreasuryCouponSecurityModel` custom type.

## Objective 1: Compute the price, visualize the cashflows for a coupon-bearing Treasury note
Unlike zero-coupon Treasury bills, which have only two cash flow events (you give money to the Treasury, and you receive the face (par) value of the `T-note` at maturity), coupon-bearing Treasury securities are more complicated because of the periodic coupon payments. Thus, it's helpful to visualize the cash flow events of notes and bonds. 
* We begin by building an instance of the `DiscreteCompoundingModel()` type and store this discount model in the `discount_model` variable:

In [2]:
discount_model = DiscreteCompoundingModel();

### TODO: Build an instance of `MyUSTreasuryCouponSecurityModel`, and compute the price
Next, let's build an instance of the `MyUSTreasuryCouponSecurityModel` type using the `build(...)` method. In this example, we'll compute the price and cashflow for a `T = 7-yr` bond, with a coupon rate of `c = 1.375%`, a yield (discount rate) `rate = 1.461%`, two coupon payments per year, i.e., $\lambda = 2$ and a face (par) value of $V_{P}$ = `100 USD`. Let's store the result in the `test_note` variable. 
* The price value reported on [TreasuryDirect.gov](https://www.treasurydirect.gov/marketable-securities/understanding-pricing/#id-for-more-detailed-formulas-and-useful-tables-264977) for this note is $V_{B}$ = `99.4299 USD`.
* Similar to zero-coupon `T-bills,` we'll use a `short-cut` syntax that relies on the [Julia piping |> operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping) and some syntax sugar to compute the coupon-bearing price, and discount factors and the cashflow for the instrument. 

In [3]:
### BEGIN SOLUTION
T = 7
r̄ = 0.01461
c = 0.01375
### END SOLUTION

test_note = build(MyUSTreasuryCouponSecurityModel, (
    T = T, rate = r̄, coupon = c, λ = 2, par = 100.0
)) |> discount_model;

We have populated the `MyUSTreasuryCouponSecurityModel` instance stored in the `test_note` variable
* Let's pull the `price,` `discount,` and `cashflow` fields from `test_note` and store them in the `nominal_computed_price,` `cashflow,` and `discount` variables

In [4]:
nominal_computed_price = test_note.price;
cashflow = test_note.cashflow;
discount = test_note.discount;
println("This computed note price = $(nominal_computed_price) USD")

This computed note price = 99.42973596186266 USD


#### Check: Are the computed and observed prices `similar`?
Use the [isapprox function](https://docs.julialang.org/en/v1/base/math/#Base.isapprox) combined with the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check the `similarity` of the computed, and observed note price. If the price values are different beyond `rtol = 1e-4`, a `false` result is generated, and an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) is thrown.

In [5]:
observed_bond_price = 99.4299;
@assert isapprox(observed_bond_price, nominal_computed_price; rtol = 1e-4)

### TODO: Visualize the `nominal,` `discounted,` and `cumulative` cash flow
Finally, let's build a table holding the `nominal,` `discounted,` and `cumulative` cash flow for this bond using the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl). We'll iterate through each period using a `for` loop and populate the `bond_data_table` variable. For each iteration of the loop, we will:
* We access values for the discount and cashflow in period `i` and compute the cumulative cashflow in the `sumvalue` variable
* We then store these data along with the nominal cash flow for each period in the `bond_data_table` array and display the data in the `bond_data_table` variable by calling the `pretty_table(...)` function (with optional values for the `header,` and `tf` arguments)

In [6]:
number_of_periods = length(cashflow)
bond_data_table = Array{Any,2}(undef, number_of_periods, 5);
sumvalue = 0.0;
for i ∈ 0:(number_of_periods - 1)
    
    discount_value = discount[i]
    payment = cashflow[i];
    sumvalue += payment;

    bond_data_table[i+1,1] = i;
    bond_data_table[i+1,2] = discount_value;
    bond_data_table[i+1,3] = discount_value*payment;
    bond_data_table[i+1,4] = payment;
    bond_data_table[i+1,5] = sumvalue;
end
pretty_table(bond_data_table; 
    header=["Period", "Discount factor", "Nominal cashflow", "Discounted cashflow", "Cumulative cashflow"], 
    tf = tf_simple)

========= ================= ================== ===================== ======================
  Period   Discount factor   Nominal cashflow   Discounted cashflow   Cumulative cashflow 
========= ================= ================== ===================== ======================
       0               1.0           -99.4297              -99.4297              -99.4297
       1            1.0073             0.6875              0.682514              -98.7472
       2           1.01466             0.6875              0.677565              -98.0697
       3           1.02208             0.6875              0.672651               -97.397
       4           1.02954             0.6875              0.667773              -96.7292
       5           1.03706             0.6875               0.66293              -96.0663
       6           1.04464             0.6875              0.658123              -95.4082
       7           1.05227             0.6875               0.65335              -94.7548
     

## Objective 2: Simulate `Theorem 4` of Malkiel for a coupon-bearing Treasury note
To simulate the asymmetry of changes in price following changes in the effective discount rate (yield), with all other values held constant, generate a new rate of the form $\bar{r}\leftarrow\beta\cdot\bar{r}$, where $\beta$ is a perturbation value; if $\beta<1$ the perturbed interest rate is _less than_ the nominal rate, if $\beta=1$ the perturbed interest rate is _equals_ the nominal rate, and if $\beta>1$ the perturbed interest rate is _greater than_ the nominal rate. For this experiment, let's use the `test_note` instance from Objective 1, let's expect change the duration from `7`-years to `20`-years.

* First, specify the number of perturbation values in the `number_of_samples_theorem_4` variable; for this simulation, let `number_of_samples_theorem_4 = 7`
    * `Note`: if `number_of_samples` is odd, you'll capture the nominal case as the center data point
* Next, specify the lower bound in the `β₁` variable and the upper bound in the `β₂` variable; let `β₁ = 0.8` and `β₂ = 1.2.`
* Finally, we compute the perturbation array (stored in the `β_theorem_4`-variable) using the [range function](https://docs.julialang.org/en/v1/base/math/#Base.range) in combination with the [Julia pipe |> operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping), and the [collect function](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}), which converts a range type to a collection, i.e., an array

In [7]:
number_of_samples_theorem_4  = 7;
β₁ = 0.8;
β₂ = 1.2;
β_theorem_4 = range(β₁, stop = β₂, length = number_of_samples_theorem_4) |> collect;

Your job is to complete the implementation of the `Theorem 4` simulation started below and analyze the simulation results. Let's display the results in a table using the `pretty_table(...)` function exported from the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl)
*  `Hint`: check out the [deepcopy function](https://docs.julialang.org/en/v1/base/copy/#Base.deepcopy) in `Julia`

In [8]:
simulation_results_thm4_array = Array{Float64,2}(undef, number_of_samples_theorem_4, 2);
for i ∈ eachindex(β_theorem_4)
    β_value = β_theorem_4[i]

    # create a copy of the test_node instance
    model = deepcopy(test_note);
    
    ### BEGIN SOLUTION
    model.rate = β_value*test_note.rate
    model.T = 20.0
    ### END SOLUTION
    
    # compute: use short-cut syntax and compute the price
    perturbed_price = model |> discount_model |> x-> x.price
    
    # capture: put data in simulation_results_thm4_array
    simulation_results_thm4_array[i,1] = 100*((model.rate - test_note.rate)/(test_note.rate));    # col 1: percentage in yield
    simulation_results_thm4_array[i,2] = 100*((model.price - test_note.price)/(test_note.price)); # col 2: percentage change in the price of the note
end
pretty_table(simulation_results_thm4_array, header=["Δr̄ (%)","ΔPrice (%)"] , tf=tf_simple)

=========== =============
    Δr̄ (%)   ΔPrice (%) 
=========== =============
     -20.0      4.26252
  -13.3333      2.50153
  -6.66667      0.77364
       0.0    -0.921818
   6.66667     -2.58548
   13.3333     -4.21796
      20.0     -5.81988
=========== =============


### Discussion questions: Do your simulation results support the `Theorem 4`?
1. From the table above, do the `Theorem 4` simulations support Malkiel's hypothesis, i.e., that the price change is asymmetric?
2. Would you expect the price asymmetry to increase or decrease with the duration of the note or bond?

## Objective 3: Simulate `Theorem 5` of Malkiel for a coupon-bearing Treasury note
To simulate the impact of changes in the discount (yield) and coupon rate on price, let's perturb the effective nominal yield $\bar{r}$, for a `high,` `nominal,` and `low` coupon rate, with all other values held constant. We'll generate a new rate of the form $\bar{r}\leftarrow\beta\cdot\bar{r}$, where $\beta$ is a perturbation value; if $\beta<1$ the perturbed interest rate is _less than_ the nominal rate, if $\beta=1$ the perturbed interest rate is _equals_ the nominal rate, and if $\beta>1$ the perturbed interest rate is _greater than_ the nominal rate.

* First, specify the number of perturbation values in the `number_of_samples_theorem_5` variable; for this simulation, let `number_of_samples_theorem_5 = 3`
    * `Note`: if `number_of_samples` is odd, we'll capture the nominal case as the center data point
* Next, specify the lower bound in the `β₁` variable and the upper bound in the `β₂` variable; let `β₁ = 0.8` and `β₂ = 1.2.`
* Finally, we compute the perturbation array (stored in the `β`-variable) using the [range function](https://docs.julialang.org/en/v1/base/math/#Base.range) in combination with the [Julia pipe |> operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping), and the [collect function](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}), which converts a range type to a collection, i.e., an array

In [9]:
number_of_samples_theorem_5 = 3;
β₁ = 0.8;
β₂ = 1.2;
β = range(β₁, stop = β₂, length = number_of_samples_theorem_5) |> collect;

Your job is to complete the implementation of the simulation of `Theorem 5` started below, and analyze the simulation results.
*  `Hint`: check out the [deepcopy function](https://docs.julialang.org/en/v1/base/copy/#Base.deepcopy) in `Julia`

In [10]:
simulation_results_thm5_array = Array{Float64,2}(undef, number_of_samples_theorem_5, number_of_samples_theorem_5);
for i ∈ eachindex(β)
    β_outer = β[i]

    # create a copy of the test_node instance
    model = deepcopy(test_note);
    
    ### BEGIN SOLUTION
    model.coupon = β_outer*test_note.coupon;
    ### END SOLUTION
    
    for j ∈ eachindex(β)
        
        β_inner = β[j];
        
        ### BEGIN SOLUTION
        model.rate = β_inner*test_note.rate;
        ### END SOLUTION

        # compute: use short-cut syntax and compute the price
        perturbed_price = model |> discount_model |> x-> x.price
        
        # compute: the percentage difference between the nominal and perturbed price
        simulation_results_thm5_array[i,j] = ((perturbed_price - nominal_computed_price)/nominal_computed_price)*100
    end
end
simulation_results_thm5_array;

Now, we'll visualize the `simulation_results_thm5_array` data array using the `pretty_table(...)` function exported from the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl).
* `NOTE`: This code assumes `number_of_samples = 3`; if you have modified the `number_of_samples` variable, you will need to update the logic that generates the table

In [11]:
# build a pretty table to display the results -
(R,C) = size(simulation_results_thm5_array)
pretty_table_data = Array{Any,2}(undef, R, C+1)

# first col holds labels -
for i ∈ 1:R
    if (i == 1)
        pretty_table_data[i,1] = "-20% coupon";
    elseif (i == 3)
        pretty_table_data[i,1] = "+20% coupon";
    else
        pretty_table_data[i,1] = "nominal coupon";
    end
end

for i = 1:R
    for j = 1:C
        pretty_table_data[i,j+1] = simulation_results_thm5_array[i,j]
    end
end

header_data = (["", "-20% yield", "nominal yield", "+20% yield"])
pretty_table(pretty_table_data, header=header_data, tf=tf_simple)

================= ============ =============== =============
                   -20% yield   nominal yield   +20% yield 
================= ============ =============== =============
     -20% coupon     0.109757        -1.83398     -3.73638
  nominal coupon      1.96352             0.0     -1.92189
     +20% coupon      3.81729         1.83398    -0.107394
================= ============ =============== =============


### Discussion question: Do your simulation results support the `Theorem 5`?
1. From the table above, do the `Theorem 5` simulations support Malkiel's hypothesis, i.e., that high coupon instruments are less sensitive to changes in yield (discount rate)?
    * `Hint` compare the center square with the corners; what do the differences in value suggest?

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.